In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage,SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
model=ChatGroq(model="openai/gpt-oss-20b",groq_api_key=os.getenv("GROQ_API_KEY"))

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Subhasish and I am a  AI Engineer")])

AIMessage(content='Hello Subhasish! 👋  \nNice to meet an AI engineer. How can I assist you today? Whether you’re working on a new model, need help with data pipelines, or just want to chat about the latest AI trends, I’m here for you.', additional_kwargs={'reasoning_content': 'The user says: "Hi , My name is Subhasish and I am a AI Engineer". Likely they want a greeting and maybe some interaction. We should respond politely, ask how we can help.'}, response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 86, 'total_tokens': 192, 'completion_time': 0.095596776, 'prompt_time': 0.005594967, 'queue_time': 0.045720852, 'total_time': 0.101191743}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_b30a5b183c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--697623af-bb65-4713-be33-22370ea70298-0', usage_metadata={'input_tokens': 86, 'output_tokens': 106, 'total_tokens': 192})

In [8]:
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi , My name is Subhasish and I am a  AI Engineer"),
        AIMessage(
            content="Hello Subhasish! 👋  \nNice to meet an AI engineer. How can I assist you today? Whether you’re working on a new model, need help with data pipelines, or just want to chat about the latest AI trends, I’m here for you"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content='You’re Subhasish, an AI Engineer.  \nThat means you design, build, and fine‑tune machine‑learning models, work on data pipelines, and help turn raw data into actionable insights—essentially turning ideas into intelligent systems.', additional_kwargs={'reasoning_content': 'The user says: "Hey What\'s my name and what do I do?" They previously said: "Hi , My name is Subhasish and I am a AI Engineer". So answer: name Subhasish, profession AI engineer. Also maybe mention what they do: building AI models, etc.'}, response_metadata={'token_usage': {'completion_tokens': 120, 'prompt_tokens': 159, 'total_tokens': 279, 'completion_time': 0.106694726, 'prompt_time': 0.010451703, 'queue_time': 0.051362757, 'total_time': 0.117146429}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_a5ac2a5d7b', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6f24b352-3a1a-43d9-91c9-c2df33306632-0', usage_metadata={'input_tokens': 159, 'output_token

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!


In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [10]:
config={"configurable":{"session_id":"chat1"}}

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Subhasish and I am a  AI Engineer")],
    config=config
)

In [12]:
response.content

'Hello Subhasish! 👋 It’s great to meet another AI Engineer. How are things going on your end? Are you working on any particular project or looking for insights on a specific AI topic? Let me know how I can help!'

In [13]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Subhasish.', additional_kwargs={'reasoning_content': 'User asked: "What\'s my name?" We should answer: "Your name is Subhasish." According to policy: we can provide the name. It\'s allowed. Provide a concise answer.'}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 149, 'total_tokens': 204, 'completion_time': 0.048763156, 'prompt_time': 0.009761229, 'queue_time': 0.046480391, 'total_time': 0.058524385}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_a5ac2a5d7b', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--ec015b9d-61c8-4c8c-860e-2e41fbbfb3cb-0', usage_metadata={'input_tokens': 149, 'output_tokens': 55, 'total_tokens': 204})

In [14]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [15]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Subhasish")]})

AIMessage(content='Hello Subhasish! 👋 How can I help you today?', additional_kwargs={'reasoning_content': 'User says "Hi My name is Subhasish". They introduced themselves. The user hasn\'t asked a question yet. We should respond politely, maybe ask how we can help.'}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 99, 'total_tokens': 158, 'completion_time': 0.052215254, 'prompt_time': 0.006440052, 'queue_time': 0.046207598, 'total_time': 0.058655306}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_a5ac2a5d7b', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--43ee6140-d4e9-4c4e-b69d-cfcecd8016b9-0', usage_metadata={'input_tokens': 99, 'output_tokens': 59, 'total_tokens': 158})

In [16]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [17]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Subhasish")],
    config=config
)


In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Subhasish.'

In [19]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [21]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Subhasish")],"language":"Bengali"})
response.content

'হ্যালো শুভাশিষ! আপনাকে স্বাগতম। কেমন আছেন?'